In [1]:
# The three obvious outliers seem to be gone, the rest of the data seems to be under a more linear relationship between the variables

# Are there any useless columns? 

df.info()

df.head()

# Can drop PID as its just an ID but we already have index as ID

df = df.drop('PID',axis=1)

len(df.columns)

# Let's find missing values

df.isnull().sum().sort_values()

(100*df.isnull().sum()/len(df)).sort_values()

# Need a function that is going to calculate percentage of null values for me so I can check 
# if altering the df changes my missing values

def percent_missing(df):
    percent_nan = 100* df.isnull().sum() / len(df)
    percent_nan = percent_nan[percent_nan>0].sort_values()
    return percent_nan

percent_nan = percent_missing(df)

plt.figure(figsize=(10,10))
sns.barplot(y=percent_nan.index,x=percent_nan)

plt.figure(figsize=(10,10))
sns.barplot(x=percent_nan,y=percent_nan.index)
plt.xlim(0,1)

percent_nan[percent_nan <1]

# how many rows are these percentages?

100/len(df)

# Is the row missing electrical the same as garage area for example?

df[df['Electrical'].isnull()]['Garage Area']

# Let's see if dropping the row that is missing electrical and garage cars will also reduce my missing value rates

df=df.dropna(axis = 0, subset = ['Electrical','Garage Cars'])

percent_nan = percent_missing(df)

percent_nan[percent_nan <1]

# Since there are so many columns with missing values where basement is mentioned maybe should look if they 
# are for the same houses.

# What if they just don't have a basement?

df[df['Bsmt Unf SF'].isnull()][['Total Bsmt SF','BsmtFin SF 1','BsmtFin SF 2','Bsmt Full Bath','Bsmt Half Bath']]

# This makes it much easier for loc 1341

# Can fill it with basement = 'None' since it's probably just missing a basement

bsmt_num_cols = ['BsmtFin SF 1', 'BsmtFin SF 2', 'Bsmt Unf SF','Total Bsmt SF', 'Bsmt Full Bath', 'Bsmt Half Bath']
df[bsmt_num_cols] = df[bsmt_num_cols].fillna(0)

bsmt_str_cols =  ['Bsmt Qual', 'Bsmt Cond', 'Bsmt Exposure', 'BsmtFin Type 1', 'BsmtFin Type 2']
df[bsmt_str_cols] = df[bsmt_str_cols].fillna('None')

percent_nan = percent_missing(df)

plt.figure(figsize=(10,10))
sns.barplot(x=percent_nan,y=percent_nan.index)
plt.xlim(0,1)

# Looks like Mas Vnr is for houses that are missing masonry vaneer so let's just fill those like we did with basement

df["Mas Vnr Type"] = df["Mas Vnr Type"].fillna("None")
df["Mas Vnr Area"] = df["Mas Vnr Area"].fillna(0)

percent_nan = percent_missing(df)

plt.figure(figsize=(10,10))
sns.barplot(x=percent_nan,y=percent_nan.index)

# Since there are no more features that are under the 1% missing data threshold I won't drop any more rows
# Garage columns all seem to be missing the same amount of data and in the description it says if they are missing data
# then the house just doesn't have a garage so let's normalize this

gar_str_cols = ['Garage Type', 'Garage Finish', 'Garage Qual', 'Garage Cond']
df[gar_str_cols] = df[gar_str_cols].fillna('None')

df['Garage Yr Blt'] = df['Garage Yr Blt'].fillna(0)

percent_nan=percent_missing(df)

plt.figure(figsize=(10,10))
sns.barplot(x=percent_nan,y=percent_nan.index)

# Will be dropping columns that are missing such a high percentage of rows like The bottom 4 here ^^

df = df.drop(['Fence','Alley','Misc Feature','Pool QC'],axis = 1)

# And let's inspect the other features

df['Fireplace Qu'].value_counts()

# Since these are just string values I will fill NA's with None again

df['Fireplace Qu'] = df['Fireplace Qu'].fillna('None')

percent_nan=percent_missing(df)
percent_nan

# All that is missing is Lot Frontage
# In this case what I can do is look at the average of this feature 
# in  the feature 'Neighborhoods' and fill missing values with the mean for that neighborhood
# This method assumes that neighborhood have pretty unifrom Lot Frontages for all the houses there
# which I am okay with since that is how it usually

df[df['Lot Frontage'].isnull()]

df.groupby('Neighborhood')['Lot Frontage'].mean()

df['Lot Frontage'] = df.groupby('Neighborhood')['Lot Frontage'].transform(lambda val: val.fillna(val.mean()))

df[df['Lot Frontage'].isnull()]['Neighborhood']

# Since GrnHill and Landmrk don't have means of their Lot Frontages I will assume they don't have them

df['Lot Frontage'] = df['Lot Frontage'].fillna(0)

percent_nan=percent_missing(df)
percent_nan

df.info()



















# Yay no more missing values!
# Now let's look at other values that may need to be split out through get_dummies or transformed

df['MS SubClass'] = df['MS SubClass'].apply(str)

nums = df.select_dtypes(exclude='object')
objs = df.select_dtypes(include='object')

objs = pd.get_dummies(objs,drop_first = True)

df= pd.concat([nums,objs],axis=1)

df.info()

# The df is finalized and ready to be worked on

X = df.drop('SalePrice',axis = 1)
y = df['SalePrice']

from sklearn.model_selection import train_test_split 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=69)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
scaled_X_train = scaler.transform(X_train)
scaled_X_test = scaler.transform(X_test)

from sklearn.linear_model import ElasticNet

base_elastic_model = ElasticNet()

params = {'alpha':[0.1,1,5,10,50,100],
              'l1_ratio':[.1, .5, .7, .9, .95, .99, 1]}

from sklearn.model_selection import GridSearchCV

grid_model = GridSearchCV(estimator = base_elastic_model, param_grid=params, cv =5, verbose = 1, scoring = 'neg_mean_squared_error')

grid_model.fit(X_train,y_train)

grid_model.best_params_

y_pred = grid_model.predict(scaled_X_test)

from sklearn.metrics import mean_absolute_error,mean_squared_error

mean_absolute_error(y_test,y_pred)

np.sqrt(mean_squared_error(y_test,y_pred))



NameError: name 'df' is not defined